# Exercise: Create Features from the application data model using Datasets

In the examples below, we create features using the `WindTurbine` application data model via the Pandas APIs provided from *C3 AI Datasets*.
The `WindTurbine.eval()` API is used to retrieve data. After retrieving the data, we resample to an hourly interval,
and create a `Feature.Set` and associated `Feature`s from the engineered features. Finally, we materialize and then retrieve the
feature data.

We follow the same path as in the in-class demo, but now we want to add a feature set `rotationSpeedFeatures`
which contains features computed from the `generatorRotationSpeed` measurement.

Code that you need to fill in will be marked with the `FIXME` tag.

## 1. Review the Application Data

First, let's review the application data. 
We can use the `eval()` method to see the data at each level of our data model. Note that `eval()` can be used to
retrieve entity fields, features, and metrics. By default, it returns the fields on the entity.

In [20]:
c3.WindTurbine.eval()

id       name location  power  manufacturer
0  demo_TURBINE-1  TURBINE-1      USA    150  demo_Siemens
1  demo_TURBINE-2  TURBINE-2      USA    200  demo_Siemens
2  demo_TURBINE-3  TURBINE-3   FRANCE    150  demo_Siemens
3  demo_TURBINE-4  TURBINE-4      USA    200       demo_GE
4  demo_TURBINE-5  TURBINE-5   FRANCE    150       demo_GE
5  demo_TURBINE-6  TURBINE-6      USA    150  demo_Siemens

In [21]:
c3.WindTurbineMeasurementSeries.eval()

earliest              latest  ...     windTurbine treatment
0 2022-01-01 00:00:02 2022-06-01 00:44:47  ...  demo_TURBINE-1      rate
1 2022-01-01 00:58:05 2022-06-01 00:36:20  ...  demo_TURBINE-2      rate
2 2022-01-01 00:15:45 2022-05-31 19:49:25  ...  demo_TURBINE-3      rate
3 2022-01-01 00:30:09 2022-06-01 00:17:41  ...  demo_TURBINE-4      rate
4 2022-01-01 01:19:39 2022-05-31 21:05:42  ...  demo_TURBINE-5      rate
5 2022-01-01 00:21:36 2022-06-01 00:46:22  ...  demo_TURBINE-6      rate

[6 rows x 5 columns]

In [22]:
c3.WindTurbineMeasurement.eval(limit=10)

start  isEstimated  ... generatorRotationSpeed activePower
0 2022-01-01 00:00:02        False  ...                 1330.0      5109.0
1 2022-01-01 00:09:12        False  ...                 1366.0      4828.0
2 2022-01-01 01:58:39        False  ...                 1369.0      4845.0
3 2022-01-01 03:11:51        False  ...                 1318.0      4762.0
4 2022-01-01 05:52:41        False  ...                 1380.0      5362.0
5 2022-01-01 06:37:04        False  ...                 1402.0      5352.0
6 2022-01-01 09:45:05        False  ...                 1324.0      4998.0
7 2022-01-01 10:28:09        False  ...                 1316.0      4431.0
8 2022-01-01 11:35:22        False  ...                 1388.0      4392.0
9 2022-01-01 11:59:28        False  ...                 1356.0      4765.0

[10 rows x 7 columns]

## 2. Retrieve, denormalize, and resample the raw time series data

To create features, we first need to retrieve the raw time series data for the fields we want (the `activePower` and `gearOilTemperature` fields of
`WindTurbineMeasurements` in this case).

In order to create features with a subject type of `WindTurbine`, we need to run our `eval()` call from that type.
In doing so, one can reference the fields from `WindTurbineMeasurement` by using the path `turbineMeasurements.data.FIELD_NAME`.

In the `eval()` call, we specify a filter to only look at `demo_TURBINE-1`. This will make our feature engineering
faster. When the actual feature data is materialized, this "subject filter" is removed.

We then call `flatten(level=2)` to denormalize the result into a flat data frame.

Finally, we will resample the unaligned data to an hourly interval.

In [10]:
# FIXME - add the generatorRotationSpeed field from the measurements entitity to the projection
proj='id, turbineMeasurements.data.start'
df = c3.WindTurbine.eval(projection=proj,
                         filter="id=='demo_TURBINE-1'")\
                   .flatten(level=2)
df.head()

subject  ... turbineMeasurements_data_generatorRotationSpeed.value
0  demo_TURBINE-1  ...                                             1330.0   
1  demo_TURBINE-1  ...                                             1366.0   
2  demo_TURBINE-1  ...                                             1369.0   
3  demo_TURBINE-1  ...                                             1318.0   
4  demo_TURBINE-1  ...                                             1380.0   

[5 rows x 3 columns]

By default, `eval` returns a `subject` column containing the Wind Turbine id for each row. Other columns are named based on the
projection expressions. We want to rename `turbineMeasurements_data_start.value` to `timestamp` so that the feature
store knows this is the column indicating the time value for each sample. We also want to provide more convenient names
for our features.

After renaming our columns, we change the types of the columns to match our desired types.
We need to do this because `flatten()` call returns Data with all columns as type `object`.

In [11]:
df = df.rename(columns={'turbineMeasurements_data_start.value':'timestamp',
                        'turbineMeasurements_data_generatorRotationSpeed.value':'generatorRotationSpeed'})\
       .astype({'timestamp':'datetime64[ns]', 'generatorRotationSpeed':'float64'})
df.head()

subject           timestamp  generatorRotationSpeed
0  demo_TURBINE-1 2022-01-01 00:00:02                  1330.0
1  demo_TURBINE-1 2022-01-01 00:09:12                  1366.0
2  demo_TURBINE-1 2022-01-01 01:58:39                  1369.0
3  demo_TURBINE-1 2022-01-01 03:11:51                  1318.0
4  demo_TURBINE-1 2022-01-01 05:52:41                  1380.0

Now, we group by the subject and resample to an hourly interval. We fill missing data with the most recent value so that
rolling feature calculations work correctly.

We will be updating the columns in our data frame, so we call `toDataRef()` to make it update-able.

In [12]:
df = df. # FIXME: add computation of the resampled df
df.head()

subject           timestamp  generatorRotationSpeed
0  demo_TURBINE-1 2022-01-01 00:00:00                  1348.0
1  demo_TURBINE-1 2022-01-01 01:00:00                  1369.0
2  demo_TURBINE-1 2022-01-01 02:00:00                  1369.0
3  demo_TURBINE-1 2022-01-01 03:00:00                  1318.0
4  demo_TURBINE-1 2022-01-01 04:00:00                  1318.0

## 3. Compute features

We can now compute our features based on this resampled time series data. We engineer a selection of features sensitive to spikes in the measurements.
For more context on these features, see the notebook `Tutorials/Use-Cases/Wind-Turbine-Predictive-Maintenance.ipynb`.

For each source data column (corresponding to a sensor), we compute:

* A rolling maximum of the difference in values using a window of size 24 (1 day) and min_periods = 1. To achieve this you would need to chain `diff()`, `rolling()` with `max()` aggregation.
* A rolling standard deviation of values with a window of size 24 and min_periods = 1 using `rolling()` with `std()` aggregation.
* A rolling mean of values with a window of size 24 and min_periods = 1 using `rolling()` with `mean()` aggregation.
* An absolute deviation from the rolling median of values with a window of size 96 and min_periods = 1 using `rolling()` with `median()` aggregation, then `.abs()` to get just the absolute value of this deviation.

Note that we can easily create several features in a loop, saving us the work of manually defining each feature indvidually.

In [13]:
sensors = ["generatorRotationSpeed"]
for sensor in sensors:
        # Create a diff column smoothed by rolling max
        df[sensor + '_diff'] = ... # FIXME compute the difference and then take the rolling maximum value over a 24 hour period
    
        # Create a rolling standard deviation column
        df[sensor + '_rolling_std'] = df[sensor].rolling(window=24, min_periods=1).std()
            # Create a rolling mean column
        df[sensor + '_rolling_mean'] = df[sensor].rolling(window=24, min_periods=1).mean()
    
        # Create a deviation from a rolling median column
        df[sensor + '_median_deviation'] = (df[sensor] - df[sensor].rolling(window=96, min_periods=1).median()).abs()

df.head()    

subject  ... generatorRotationSpeed_median_deviation
0  demo_TURBINE-1  ...                                     0.0
1  demo_TURBINE-1  ...                                    10.5
2  demo_TURBINE-1  ...                                     0.0
3  demo_TURBINE-1  ...                                    40.5
4  demo_TURBINE-1  ...                                    30.0

[5 rows x 7 columns]

Drop the original data values, since we only need the computed features.

In [14]:
df = df.drop(sensors, axis=1)
df.head()

subject  ... generatorRotationSpeed_median_deviation
0  demo_TURBINE-1  ...                                     0.0
1  demo_TURBINE-1  ...                                    10.5
2  demo_TURBINE-1  ...                                     0.0
3  demo_TURBINE-1  ...                                    40.5
4  demo_TURBINE-1  ...                                    30.0

[5 rows x 6 columns]

We now have our final engineered feature data. Show the final columns we have in our `Data` instance.

In [15]:
df.infer_dtypes

subject                                            object
timestamp                                  datetime64[ns]
generatorRotationSpeed_diff                       float64
generatorRotationSpeed_rolling_std                float64
generatorRotationSpeed_rolling_mean               float64
generatorRotationSpeed_median_deviation           float64
dtype: object

## 4. Create and materialize a feature set

We can now create a feature set and the underlying features via `upsertFeatureSet`.
We specify `materialize=False`, as we want to do that separately. Since we know the interval of these
features is hourly, we also set that on the feature set.

In [16]:
fs = df.upsertFeatureSet(...) # FIXME: add parameters to upsertFeatureSet
fs.get() # show the full feature set definition

{
  "type" : "Feature.Set",
  "id" : "WindTurbine#rotationSpeedFeatures",
  "meta" : {
    "appCode" : 1761561475578381880,
    "env" : "c3",
    "app" : "wt10",
    "created" : "2023-03-28T00:26:41Z",
    "createdBy" : "BA",
    "updated" : "2023-03-28T00:26:41Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-03-28T00:26:41Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature.Set"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "interval" : "HOUR",
  "features" : [ "generatorRotationSpeed_diff", "generatorRotationSpeed_rolling_std", "generatorRotationSpeed_rolling_mean", "generatorRotationSpeed_median_deviation" ]
}

We materialize the feature set. This will materialize for all subjects, not just for `demo_TURBINE-1`.

In [17]:
# FIXME add materialize call here

## 5. Evaluate feature data

To retrieve feature data from the feature store, you can use `WindTurbine.evalFeatureSetBatch`.
First, we look at Turbine 1, which was used in our feature engineering.

In [18]:
eval_feature_result1 = c3.WindTurbine.evalFeatureSetBatch(...) # FIXME add parameters
eval_feature_result1.head()

subject  ... generatorRotationSpeed_rolling_std
0  demo_TURBINE-1  ...                                NaN
1  demo_TURBINE-1  ...                          14.849242
2  demo_TURBINE-1  ...                          12.124356
3  demo_TURBINE-1  ...                          24.124676
4  demo_TURBINE-1  ...                          25.579288

[5 rows x 6 columns]

Now, we look at Turbine 2, which was not included in the original feature engineering, but was included in materialization.

In [19]:
eval_feature_result2 = c3.WindTurbine.evalFeatureSetBatch(...) # FIXME add parameters
eval_feature_result2.head()

subject  ... generatorRotationSpeed_rolling_std
0  demo_TURBINE-2  ...                         112.845797
1  demo_TURBINE-2  ...                         150.033686
2  demo_TURBINE-2  ...                         170.952171
3  demo_TURBINE-2  ...                         187.044572
4  demo_TURBINE-2  ...                         194.602321

[5 rows x 6 columns]